###***Dependencies***


In [ ]:
import tensorflow as tf
from statistics import mean, stdev
from operator import itemgetter
import json

###***Dataset contains code snippets of these 30 progamming languages for Training***



In [ ]:
with open('languages.json') as f:
  language_info = json.load(f)

language_info

{'Batchfile': ['bat', 'cmd', 'btm'],
 'C': ['c'],
 'C#': ['cs'],
 'C++': ['cpp', 'cc'],
 'CSS': ['css'],
 'CoffeeScript': ['coffee', 'litcoffee'],
 'Erlang': ['erl', 'hrl'],
 'Go': ['go'],
 'HTML': ['html'],
 'Haskell': ['hs', 'lhs'],
 'Java': ['java'],
 'JavaScript': ['js', 'es6'],
 'Jupyter Notebook': ['ipynb'],
 'Lua': ['lua'],
 'Markdown': ['md'],
 'Matlab': ['matlab', 'm'],
 'Objective-C': ['mm', 'm'],
 'PHP': ['php'],
 'Perl': ['pl', 'pm'],
 'PowerShell': ['ps1'],
 'Python': ['py'],
 'R': ['r', 'rdata', 'rds', 'rda'],
 'Ruby': ['rb'],
 'Rust': ['rs'],
 'SQL': ['sql'],
 'Scala': ['scala'],
 'Shell': ['sh'],
 'Swift': ['swift'],
 'TeX': ['tex'],
 'TypeScript': ['ts', 'tsx']}

###***Mapping***


In [ ]:
language_map = { name: exts[0] for name, exts in language_info.items() }

extension_map = { ext: name for name, ext in language_map.items() }

###***Load Model***

In [ ]:
saved_model =tf.saved_model.load("model")

###***Prediction***


In [ ]:
def language_used(text):
  if not text.strip():
    return 'Please input code snippet.'

  content_tensor = tf.constant([text])
  predicted = saved_model.signatures['serving_default'](content_tensor)

  numpy_floats = predicted['scores'][0].numpy()
  extensions = predicted['classes'][0].numpy()

  probability_values = (float(value) for value in numpy_floats)  

  languages = ( extension_map[ext.decode()] for ext in extensions)
  unsorted_scores = zip(languages, probability_values)
  language_probabilities = sorted(unsorted_scores, key=itemgetter(1), reverse=True)
  probabilities = [value for _, value in language_probabilities]

  if not _is_reliable(probabilities):
    print('No any programming language detected')
    return 'More Training to be done on large dataset.'

  language_name, _ = language_probabilities[0]
  return language_name

In [ ]:
def _is_reliable(probabilities) -> bool:
        """Arbitrary rule to determine if the prediction is reliable:
        The predicted language probability must be higher than
        2 times standard deviations from the mean.
        """
        threshold = mean(probabilities) + 2*stdev(probabilities)
        predicted_language_probability = max(probabilities)
        return predicted_language_probability > threshold

##***Results***





In [ ]:
text1 = """ 
package main
import "fmt"

func main() {
    fmt.Println("My mascot is a gopher and Google loves me. Who am I?")
}
"""

language_used(text1)

'Go'

In [ ]:
text2 = """

# Store input numbers

num1 = input('Enter first number: ')

num2 = input('Enter second number: ')

 

# Add two numbers

sum = float(num1) + float(num2)

 

# Display the sum

print('The sum of {0} and {1} is {2}'.format(num1, num2, sum))
"""

language_used(text2)

'Python'

In [ ]:
text3 = """
// This ia a sample getter function

private int LOC = 0;

 
public int getLOC() {

                                return LOC;

                }

public void setLOC(int lOC) {

                                LOC = lOC;

                }

 """   
 
language_used(text3)

'Java'

In [ ]:
text4 = """

/** Turn command line arguments to uppercase */
object Main {
  def main(args: Array[String]) {
    val res = for (a <- args) yield a.toUpperCase
    println("Arguments: " + res.toString)
  }
}

"""

language_used(text4)

'Scala'

In [ ]:
text = """ 

/bin/which 

"""
language_used(text)

'Shell'

In [ ]:
text5 = """
interface User {
  name: string;
  id: number;
}

class UserAccount {
  name: string;
  id: number;

  constructor(name: string, id: number) {
    this.name = name;
    this.id = id;
  }
}

const user: User = new UserAccount("Murphy", 1);
"""

language_used(text5)

'TypeScript'